In [1]:
#!pip install mediapipe==0.10.8 --user

In [2]:
import math
import cv2
import mediapipe as mp
import time

In [3]:
class detectormanos():
    #Inicializar parametros de detección
    def __init__(self, mode=False, maxManos = 2, model_complexity=1, Confdeteccion = 0.5, Confsegui = 0.5):
        self.mode = mode          #Creamos el objeto y el tendra su propia variable
        self.maxManos = maxManos  #Lo mismo haremos con todos los objetos
        self.compl = model_complexity
        self.Confdeteccion = Confdeteccion
        self.Confsegui = Confsegui

        #Parametros que dibujan y detectan mano
        self.mpmanos = mp.solutions.hands #Detecta la mano
        self.manos = self.mpmanos.Hands(self.mode, self.maxManos, self.compl, self.Confdeteccion, self.Confsegui)#Bajo estos parametros
        self.dibujo = mp.solutions.drawing_utils #Dibuja lo que necesitamos
        self.tip = [4,8,12,16,20]  #son los puntos claves para detectar las puntas de los dedos

 #Con esta funcion tomamos frame para detectar la mano en ese instante---
    def encontrarmanos(self, frame, dibujar = True ):
        imgcolor = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        self.resultados = self.manos.process(imgcolor) #Hacemos la deteccion en el frame que mandamos

        if self.resultados.multi_hand_landmarks: #Observamos si hay algo en la deteccion
            for mano in self.resultados.multi_hand_landmarks:  #Si hay entramos a este for (Nuestro maximo de manos es 2)
                if dibujar:
                    self.dibujo.draw_landmarks(frame, mano, self.mpmanos.HAND_CONNECTIONS)  # Dibujamos las conexiones de los puntos
        return frame

  #Guardamos coordenadas x y de todos los puntos de las manos
    def encontrarposicion(self, frame, ManoNum = 0, dibujar = True, color = []):
        xlista = []
        ylista = []
        bbox = []
        player = 0
        self.lista = []  #lista para guardar las coordenadas
        if self.resultados.multi_hand_landmarks:#Si detectamos algo 
            miMano = self.resultados.multi_hand_landmarks[ManoNum]
            prueba = self.resultados.multi_hand_landmarks
            player = len(prueba)
 
            for id, lm in enumerate(miMano.landmark):
                alto, ancho, c = frame.shape   #Extraemos dimensiones de los frames
                cx, cy = int(lm.x * ancho), int(lm.y * alto)  # Convertimos la informacion en pixeles
                xlista.append(cx)
                ylista.append(cy)
                self.lista.append([id, cx, cy]) #Guardamos en la lista las coordenadas en pixeles de x e y, de cada nodo de la mano id=Nodo
                if dibujar:
                    cv2.circle(frame,(cx, cy), 3, (0, 0, 0), cv2.FILLED)  # Dibujamos un circulo en cada nodo

            xmin, xmax = min(xlista), max(xlista) #Tomamos los valores maximos y minimos de x 
            ymin, ymax = min(ylista), max(ylista) #Tomamos los valores maximos y minimos de  y
            bbox = xmin, ymin, xmax, ymax #Con esas coordenadas hacemos una caja
            if dibujar:
                # Con base a eso dibujamos un cuadro
                cv2.rectangle(frame,(xmin - 20, ymin - 20), (xmax + 20, ymax + 20), color,2)
        return self.lista, bbox, player